In [ ]:
# This jupyter lab allows users to create the final database needed with a single run. 
# Users can also choose to import database using the final_db.sql file provided. 

In [ ]:
# Create a database named final and create the first table named pop_superpop
import pymysql
import mysql.connector
from mysql.connector import Error
from mysql.connector import errorcode
import pandas as pd
import zarr
import time
import re

# Specify the database information here
db_name = "final"
db_host = "localhost"
db_username = "root"
db_password = "Teammint123@"

try:
    conn = pymysql.connect(host=db_host,
                           port=int(3306),
                           user=db_username,
                           password=db_password)
    cursor = conn.cursor()

    # SQL commands for creating database and first table pop_superpop
    sql_commands = [
        "CREATE DATABASE IF NOT EXISTS final;",
        "USE final;",
        "CREATE TABLE IF NOT EXISTS pop_superpop (sample_id VARCHAR(255) NOT NULL, population VARCHAR(255), superpopulation VARCHAR(255), PRIMARY KEY (sample_id));"
    ]

    # Execute SQL commands
    for command in sql_commands:
        # Reconnect if cursor is not connected
        if not cursor.connection:
            cursor = conn.cursor()
        cursor.execute(command)

except pymysql.Error as e:
    print("Error:", e)

finally:
    if conn:
        conn.close()


In [ ]:
# Populating data for table pop_superpop here.

# change the path of pop_superpop.tsv file if needed
superpop = pd.read_csv("pop_superpop.tsv", sep='\t')

try:
    # connect to the mysql server
    connection = mysql.connector.connect(user= db_username,
                                         password= db_password,
                                         host= db_host,
                                         database= db_name)
    if connection.is_connected:
        print("connection succeeded")
        cursor = connection.cursor()
        # specify table name here
        table_name = 'pop_superpop'
        # specify table name here, for example: 'sample_id, population, superpopulation'
        column_names = 'sample_id, population, superpopulation'
        batch_size = 1000
        # Use executemany to insert data in batches
        for i in range(0, len(superpop), batch_size):
            batch_data = superpop.iloc[i:i + batch_size].values.tolist()
            query = f"INSERT INTO {table_name} ({column_names}) VALUES ({', '.join(['%s'] * len(superpop.columns))})"
            cursor.executemany(query, batch_data)
            connection.commit()

        print('records inserted successfully!')

except Error as err:
    # check connection error
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Username or password isnt correct")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
finally:
    # close connection in the end
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection closed.")

In [ ]:
# Create the second table named pca_data here

try:
    conn = pymysql.connect(host=db_host,
                           port=int(3306),
                           user=db_username,
                           password=db_password,
                           db=db_name)
    cursor = conn.cursor()

    # SQL commands for creating second table pca_data
    sql_commands = [
        "CREATE TABLE IF NOT EXISTS pca_data (sample_id VARCHAR(255) NOT NULL, pc1 FLOAT, pc2 FLOAT, PRIMARY KEY (sample_id), FOREIGN KEY (sample_id) REFERENCES pop_superpop (sample_id));"
    ]

    # Execute SQL commands
    for command in sql_commands:
        # Reconnect if cursor is not connected
        if not cursor.connection:
            cursor = conn.cursor()
        cursor.execute(command)

except pymysql.Error as e:
    print("Error:", e)

finally:
    if conn:
        conn.close()


In [ ]:
# Populating data for table pca_data here, change the path of pca_data.csv if needed.
clustering_data = pd.read_csv("pca_data.csv", sep=',')

try:
    # connect to the mysql server
    connection = mysql.connector.connect(user= db_username,
                                         password= db_password,
                                         host= db_host,
                                         database= db_name)
    if connection.is_connected:
        print("connection succeeded")
        cursor = connection.cursor()
        # specify table name here
        table_name = 'pca_data'
        # specify table name here, for example: 'sample_id, population, superpopulation'
        column_names = 'sample_id, pc1, pc2'
        batch_size = 1000
        # Use executemany to insert data in batches
        for i in range(0, len(clustering_data), batch_size):
            batch_data = clustering_data.iloc[i:i + batch_size].values.tolist()
            query = f"INSERT INTO {table_name} ({column_names}) VALUES ({', '.join(['%s'] * len(clustering_data.columns))})"
            cursor.executemany(query, batch_data)
            connection.commit()

        print('records inserted successfully!')

except Error as err:
    # check connection error
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Username or password isnt correct")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
finally:
    # close connection in the end
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection closed.")

In [ ]:
# Create the third table named admixture_data here

try:
    conn = pymysql.connect(host=db_host,
                           port=int(3306),
                           user=db_username,
                           password=db_password,
                           db=db_name)
    cursor = conn.cursor()

    # SQL command for creating table admixture_data
    sql_command = """
        CREATE TABLE IF NOT EXISTS admixture_data (
            sample_id VARCHAR(255) NOT NULL,
            pop1 FLOAT,
            pop2 FLOAT,
            pop3 FLOAT,
            pop4 FLOAT,
            pop5 FLOAT,
            assignment VARCHAR(255),
            PRIMARY KEY (sample_id),
            FOREIGN KEY (sample_id) REFERENCES pop_superpop (sample_id)
        );
    """

    # Execute SQL command
    cursor.execute(sql_command)

    print("Table 'admixture_data' created successfully.")

except pymysql.Error as e:
    print("Error:", e)

finally:
    if conn:
        conn.close()


In [ ]:
# Populating table admixture_data here

# Specify the path to df_sorted_q.csv here if needed 
admixture_data = pd.read_csv("df_sorted_q.csv", sep=',')

try:
    # connect to the mysql server
    connection = mysql.connector.connect(user= db_username,
                                         password= db_password,
                                         host= db_host,
                                         database= db_name)
    if connection.is_connected():
        print("Connection succeeded")
        cursor = connection.cursor()
        # specify table name here
        table_name = 'admixture_data'
        # specify table name here, for example: 'sample_id, population, superpopulation'
        column_names = 'sample_id, pop1, pop2, pop3, pop4, pop5, assignment'
        batch_size = 1000
        total_records = len(admixture_data)
        print(len(admixture_data))
        records_inserted = 0
        # Use executemany to insert data in batches
        for i in range(0, total_records, batch_size):
            print(i)
            batch_data = admixture_data.iloc[i:i + batch_size].values.tolist()
            query = f"INSERT INTO {table_name} ({column_names}) VALUES ({', '.join(['%s'] * len(admixture_data.columns))})"
            cursor.executemany(query, batch_data)
            connection.commit()
            records_inserted += len(batch_data)
            print(f"{records_inserted}/{total_records} records inserted")

        print('Records inserted successfully!')

except mysql.connector.Error as err:
    # check connection error
    if err.errno == mysql.connector.errorcode.ER_ACCESS_DENIED_ERROR:
        print("Username or password isn't correct")
    elif err.errno == mysql.connector.errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
finally:
    # close connection in the end
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection closed.")


In [ ]:
# Create table named snp_char here

try:
    conn = pymysql.connect(host=db_host,
                           port=int(3306),
                           user=db_username,
                           password=db_password,
                           db=db_name)
    cursor = conn.cursor()

    # SQL commands for creating table snp_char and INDEX for column position
    sql_commands = [
        "SELECT * FROM admixture_data ORDER BY assignment ASC;",
        "CREATE TABLE IF NOT EXISTS snp_char (snp_id VARCHAR(255) NOT NULL, position INT, ref_base CHAR(1), alt_base CHAR(1), disease_name LONGTEXT, classification LONGTEXT, PRIMARY KEY (snp_id));",
        "CREATE INDEX idx_position ON snp_char (position);"]

    # Execute SQL commands
    for command in sql_commands:
        # Reconnect if cursor is not connected
        if not cursor.connection:
            cursor = conn.cursor()
        cursor.execute(command)

except pymysql.Error as e:
    print("Error:", e)

finally:
    if conn:
        conn.close()


In [ ]:
# Populating table snp_char here

# Read the zarr file here, change the path of zarr file if needed
zarr_path = 'annotated.zarr'
callset_h1k = zarr.open_group(zarr_path, mode='r')

# Read the required columns from zarr file here
pos = callset_h1k['1/variants/POS']
alt = callset_h1k["1/variants/ALT"]
ref = callset_h1k['1/variants/REF']
snp_id = callset_h1k["1/variants/ID"]

# Change the snp ID like "rsXXXX:XXXXX:X:X" to "rsXXXX"
snp_id = [element.split(';')[0] if element.startswith('rs') else element for element in snp_id]

data_to_insert = [
    (SNP_ID, int(position), reference, alternate) for (SNP_ID, position, reference, alternate) in
    zip(snp_id, pos, ref, alt)
]

try:
    # connect to the mysql server
    connection = mysql.connector.connect(user= db_username,
                                         password= db_password,
                                         host= db_host,
                                         database= db_name)
    if connection.is_connected:
        print("connection succeeded")
        cursor = connection.cursor()
        # specify table name here
        table_name = 'snp_char'
        # specify table name here, for example: 'sample_id, population, superpopulation'
        column_names = 'snp_id, position, ref_base, alt_base'
        query = f"INSERT INTO {table_name} ({column_names}) VALUES ({', '.join(['%s'] * 4)})"

        # Use executemany to insert data in batches
        batch_size = 1000
        for i in range(0, len(data_to_insert), batch_size):
            start = time.time()
            batch = data_to_insert[i:i + batch_size]
            cursor.executemany(query, batch)
            connection.commit()
            end = time.time()
            print(f"{i + batch_size} of lines added, time cost: {round(end-start, 2)} s")

        print('records inserted successfully!')

except Error as err:
    # check connection error
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Username or password isnt correct")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
finally:
    # close connection in the end
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection closed.")

In [ ]:
# Create table named gene_names here

try:
    conn = pymysql.connect(host=db_host,
                           port=int(3306),
                           user=db_username,
                           password=db_password,
                           db=db_name)
    cursor = conn.cursor()

    # SQL command for creating table gene_names
    sql_command = """
        CREATE TABLE gene_names (
            id INT NOT NULL auto_increment,
            gene_name VARCHAR(255) NOT NULL,
            position INT,
            PRIMARY KEY (id),
            FOREIGN KEY (position) REFERENCES snp_char (position)
        );
    """

    # Execute SQL command
    cursor.execute(sql_command)

    print("Table 'gene_names' created successfully.")

except pymysql.Error as e:
    print("Error:", e)

finally:
    if conn:
        conn.close()


In [ ]:

# This function creates the data list used for populating table gene_names
def gene_data_insert():
    
    # Change the path if needed
    callset_path = 'annotated.zarr'
    csv_file_path = "only_gene_names.csv"
    
    print("Reading gene names from CSV file...")
    gene_name_df = pd.read_csv(csv_file_path, usecols=['Gene name', 'Gene start (bp)', 'Gene end (bp)'])

    print("Reading data from Zarr file...")
    callset = zarr.open_group(callset_path, mode='r')
    pos_data = callset['1/variants/POS'][:]

    print("Mapping SNPs to genes with logging...")
    start_time = time.time()
    data_to_insert = []

    snp_df = pd.DataFrame(pos_data, columns=['POS'])

    for index, row in gene_name_df.iterrows():
        gene_name = row['Gene name']
        start_bp = row['Gene start (bp)']
        end_bp = row['Gene end (bp)']

        # Match snps to genes based on snps' positions and genes' start and end base
        matched_snps = snp_df[(snp_df['POS'] >= start_bp) & (snp_df['POS'] <= end_bp)]
        print(f"Gene: {gene_name}, Start: {start_bp}, End: {end_bp}, Matched SNPs: {len(matched_snps)}")

        for snp in matched_snps['POS']:
            data_to_insert.append((gene_name, snp))

    end_time = time.time()
    print(f"Total processing time: {end_time - start_time} seconds")

    return data_to_insert

In [ ]:
# Populating table gene_names here:

start0 = time.time()
data_to_insert = gene_data_insert()
end0 = time.time()
print(f"data_to_insert ready, time cost: {round(end0-start0, 2)} s")

try:
    # connect to the mysql server
    connection = mysql.connector.connect(user= db_username,
                                         password= db_password,
                                         host= db_host,
                                         database= db_name)
    if connection.is_connected:
        print("connection succeeded")
        cursor = connection.cursor()
        # specify table name here
        table_name = 'gene_names'
        # specify table name here, for example: 'sample_id, population, superpopulation'
        column_names = 'gene_name, position'
        query = f"INSERT INTO {table_name} ({column_names}) VALUES ({', '.join(['%s'] * 2)})"

        # Use executemany to insert data in batches
        batch_size = 1000
        for i in range(0, len(data_to_insert), batch_size):
            start = time.time()
            batch = data_to_insert[i:i + batch_size]
            cursor.executemany(query, batch)
            connection.commit()
            end = time.time()
            print(f"{i + batch_size} of lines added, time cost: {round(end-start, 2)} s")

        print('records inserted successfully!')

except Error as err:
    # check connection error
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Username or password isnt correct")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
finally:
    # close connection in the end
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection closed.")

In [ ]:
# Create table named allele_freq here

try:
    conn = pymysql.connect(host=db_host,
                           port=int(3306),
                           user=db_username,
                           password=db_password,
                           db=db_name)
    cursor = conn.cursor()

    # SQL commands for creating table allele_freq
    sql_commands = """
    CREATE TABLE IF NOT EXISTS allele_freq (
        snp_id VARCHAR(255) NOT NULL, 
        SIB FLOAT, GBR FLOAT, FIN FLOAT, CHS FLOAT, PUR FLOAT, CDX FLOAT, 
        CLM FLOAT, IBS FLOAT, PEL FLOAT, PJL FLOAT, KHV FLOAT, ACB FLOAT, 
        GWD FLOAT, ESN FLOAT, BEB FLOAT, MSL FLOAT, STU FLOAT, ITU FLOAT, 
        CEU FLOAT, YRI FLOAT, CHB FLOAT, JPT FLOAT, LWK FLOAT, ASW FLOAT, 
        MXL FLOAT, TSI FLOAT, GIH FLOAT, 
        PRIMARY KEY (snp_id), FOREIGN KEY (snp_id) REFERENCES snp_char (snp_id)
        );
    """
    

    # Execute SQL commands
    for command in sql_commands:
        # Reconnect if cursor is not connected
        if not cursor.connection:
            cursor = conn.cursor()
        cursor.execute(command)

except pymysql.Error as e:
    print("Error:", e)

finally:
    if conn:
        conn.close()


In [ ]:
# Populating table allele_freq:

# Specify the path to allele_freq_df.csv file if needed
allele_freq_df = pd.read_csv("allele_freq_df.csv", sep=',')
# Fill the NA/NaN values with "0"
allele_freq_df = allele_freq_df.fillna(0)

try:
    # connect to the mysql server
    connection = mysql.connector.connect(user= db_username,
                                         password= db_password,
                                         host= db_host,
                                         database= db_name)
    if connection.is_connected:
        print("connection succeeded")
        cursor = connection.cursor()
        # specify table name here
        table_name = 'allele_freq'
        # specify table name here, for example: 'sample_id, population, superpopulation'
        column_names = 'snp_id, SIB, GBR, FIN, CHS, PUR, CDX, CLM, IBS, PEL, PJL, KHV,'\
        ' ACB, GWD, ESN, BEB, MSL, STU, ITU, CEU, YRI, CHB, JPT, LWK, ASW, MXL, TSI, GIH'
        batch_size = 1000
        # Use executemany to insert data in batches
        for i in range(0, len(allele_freq_df), batch_size):
            batch_data = allele_freq_df.iloc[i:i + batch_size].values.tolist()
            query = f"INSERT INTO {table_name} ({column_names}) VALUES ({', '.join(['%s'] * len(allele_freq_df.columns))})"
            cursor.executemany(query, batch_data)
            connection.commit()

        print('records inserted successfully!')

except Error as err:
    # check connection error
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Username or password isnt correct")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
finally:
    # close connection in the end
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection closed.")

In [ ]:
# This function creates the data list used for populating clinical information into table snp_char
def data_inserting():
    # Change the path if needed
    clinvar_zarr_path = 'clinvar.zarr'
    callset_cli = zarr.open_group(clinvar_zarr_path, mode='r')
    cli_pos = callset_cli['1/variants/POS']
    cli_alt = callset_cli['1/variants/ALT']
    cli_ref = callset_cli['1/variants/REF']
    disease_name = callset_cli['1/variants/CLNDN']
    classification = callset_cli['1/variants/CLNSIG']
    # We use the position, reference base and alternate base information of each snp
    # from clinvar.vcf to match the snps from chr1.vcf
    compare_data = [(x1, x2, x3) for x1, x2, x3 in zip(cli_pos, cli_ref, cli_alt)]
    clinical_data = [[x1, x2, x3, x4, x5] for x1, x2, x3, x4, x5 in
                     zip(disease_name, classification, cli_pos, cli_ref, cli_alt)]

    zarr_path = 'annotated.zarr'
    callset_h1k = zarr.open_group(zarr_path, mode='r')
    snp_pos = callset_h1k['1/variants/POS']
    snp_alt = callset_h1k['1/variants/ALT']
    snp_ref = callset_h1k['1/variants/REF']
    table_data = [(x1, x2, x3) for x1, x2, x3 in zip(snp_pos, snp_ref, snp_alt)]
    table_data = set(table_data)

    # Only the clinical information of matched snps will be inserted into database 
    element_pos = [index for index, element in enumerate(compare_data)
                   if element in table_data]
    insert_data = [clinical_data[i] for i in element_pos]
    return insert_data

In [ ]:
# Populating column "disease_name" and "classification" in table snp_char here.

insert_data = data_inserting()
list_of_dicts = [{'name': sublist[0], 'classif': sublist[1], 'pos': int(sublist[2]), 'ref': sublist[3],
                  'alt': sublist[4]} for sublist in insert_data]

try:
    # connect to the mysql server
    connection = mysql.connector.connect(user="root",
                                         password="Teammint123@",
                                         host='localhost',
                                         database='final')
    if connection.is_connected:
        print("connection succeeded")
        cursor = connection.cursor()
        # specify query here
        query = f"UPDATE snp_char SET disease_name = %(name)s, classification = %(classif)s" \
                f"WHERE position = %(pos)s AND ref_base = %(ref)s AND alt_base = %(alt)s"

        start = time.time()
        cursor.executemany(query, list_of_dicts)
        connection.commit()
        end = time.time()
        print(f'records inserted successfully!, time cost: {round(end-start, 2)} s')

except Error as err:
    # check connection error
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Username or password isnt correct")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
finally:
    # close connection in the end
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection closed.")

In [ ]:
# This part of codes check if the snp id from annotated.zarr and allele_freq_df is matching to each other. 
import zarr
import pandas as pd


zarr_path = 'annotated.zarr' 
callset_h1k = zarr.open_group(zarr_path, mode='r')
snp_id = callset_h1k["1/variants/ID"]
# change the snp ids from annotated.zarr, example: rsXXXX:XXXX:X:X to rsXXXX
list1 = [element.split(';')[0] if element.startswith('rs') else element for element in snp_id]
# get the snp ids from allele_freq dataframe
allele_freq_df = pd.read_csv("allele_freq_df.csv", sep=',')
list2 = allele_freq_df.iloc[:, 0].tolist()
# check if the snp ids from both dataframe is the same. if some of them are not, print the snp_ID which are different 
set1 = set(list1)
filtered = [element for element in list2 if element not in set1]
print(filtered)
# If started fully from scrath 4 Id will not match - can change manually in the allele_freq_df.csv
